## Launching Spark

In [1]:
%env SPARK_HOME=/home/ggomarr/.spark
master_node='spark://Otter:7077'
app_name='ZZ_Spark'
dataframeMode=True
stopSpark=False

env: SPARK_HOME=/home/ggomarr/.spark


In [2]:
!/home/ggomarr/.spark/sbin/start-master.sh
!/home/ggomarr/.spark/sbin/start-slave.sh {master_node}

org.apache.spark.deploy.master.Master running as process 3733.  Stop it first.
org.apache.spark.deploy.worker.Worker running as process 3793.  Stop it first.


In [3]:
import findspark
findspark.init()

In [4]:
if dataframeMode:
    from pyspark.sql import SparkSession
    spark=SparkSession.builder.master(master_node).appName(app_name).getOrCreate()
    sc=spark.sparkContext
else:
    from pyspark import SparkConf, SparkContext
    conf=SparkConf().setMaster(master_node).setAppName(app_name)
    sc=SparkContext(conf=conf)

## Test using a text file

In [5]:
def process_row(row):
    row=row.strip()
    return [ (len(row),(row)) ] if row else []

file_rows=(sc.textFile("file:///home/ggomarr/.spark/NOTICE")
                      .flatMap(process_row))

for r in file_rows.takeOrdered(5,key=lambda x: -x[0]):
    print(r)

(183, u'(Common Development and Distribution License (CDDL) v1.0) JavaBeans Activation Framework (JAF) (javax.activation:activation:1.1 - http://java.sun.com/products/javabeans/jaf/index.jsp)')
(166, u'(COMMON DEVELOPMENT AND DISTRIBUTION LICENSE (CDDL) Version 1.0) (GNU General Public Library) Streaming API for XML (javax.xml.stream:stax-api:1.0-2 - no url defined)')
(142, u'(CDDL 1.0) Servlet Specification 2.5 API (org.mortbay.jetty:servlet-api-2.5:6.1.14 - http://jetty.mortbay.org/project/modules/servlet-api-2.5)')
(126, u'The following components are provided under the Common Development and Distribution License 1.0. See project link for details.')
(126, u'The following components are provided under the Common Development and Distribution License 1.1. See project link for details.')


## Enjoy

## Clean up

In [8]:
sc.stop()

In [ ]:
if stopSpark:
    !/home/ggomarr/.spark/sbin/stop-slave.sh
    !/home/ggomarr/.spark/sbin/stop-master.sh